In [1]:
import akasha
from akasha_plus.agents.tools import webpage_summary_tool, webpage_summary_func
from time import time
import concurrent.futures

/home/ychm0304/anaconda3/envs/test/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = "openai:gpt-3.5-turbo"
result = {}
contexts = []
report = {}
response = ""

In [3]:
QA = akasha.Doc_QA(model=model, max_doc_len=8000)
summary_class = akasha.Summary(chunk_size=1000, max_doc_len=7000)
# agent = akasha.test_agent(verbose=True, tools=[webpage_summary_tool], model='openai-gpt-4')

In [4]:
titles = {
    "全球電池市場概況": {
        "市場規模與增長趨勢",
        "主要市場區域分析",
        "主要企業概況",
    },
    "技術發展趨勢": {
        "鋰離子電池技術進展",
        "固態電池技術",
        "其他新興電池技術",
    },
    "應用領域與市場機會": {
        "電動車市場",
        "消費電子產品",
        "能源儲存系統",
        "工業及其他應用",
        "市場需求預測",
        "市場挑戰與機遇",
    },
    "政策與供應鏈分析": {
        "各國政策支持與法規要求",
        "環保與回收政策",
        "供應鏈與原材料供應",
        "產業佈局與投資趨勢",
    }
}

links = [
    "https://www.maonline.com.tw/article_inside.php?i=798",
    "https://www.materialsnet.com.tw/DocView.aspx?id=53579",
    "https://ic.tpex.org.tw/introduce.php?ic=E000",
    "https://ieknet.iek.org.tw/ieknews/news_open.aspx?actiontype=ieknews&indu_idno=5&nsl_id=1039fffa4a7d408cb3f5c579762637e1",
    "https://yez.one/post/batterycell",
    "https://www.thenewslens.com/article/181546"
]

# for i, title in enumerate(titles):
#     for j, subtitle in enumerate(title):
#         if j == 0:
#             title[j] = f"主標題{i + 1}: " + title[j]
#         else:
#             title[j] = f"次標題{j}: " + title[j]


theme = "電池產業的發展趨勢"
# titles = ["企業前瞻"]

print(titles)

{'全球電池市場概況': {'主要企業概況', '主要市場區域分析', '市場規模與增長趨勢'}, '技術發展趨勢': {'固態電池技術', '鋰離子電池技術進展', '其他新興電池技術'}, '應用領域與市場機會': {'市場挑戰與機遇', '工業及其他應用', '電動車市場', '能源儲存系統', '市場需求預測', '消費電子產品'}, '政策與供應鏈分析': {'各國政策支持與法規要求', '供應鏈與原材料供應', '產業佈局與投資趨勢', '環保與回收政策'}}


In [ ]:
start_time = time()

for title in titles:
    for link in links:
        # question = f"幫我從網站: {link} 中整理出有關{theme}的文字，必須依照以下的標題結構: {title}，字數約250字，將list_result設為false。"

        print("##################################")
        print("Summarizing website: ", link)
        print("##################################")

        contexts.append(
            # agent(question, messages=[])
            webpage_summary_func(
                url=link,
                summary_len=500,
                extract_topics=title,
                list_result=False,
            )
        )

    response = QA.ask_self(
        prompt=f"將此內容以客觀角度進行融合，避免使用\"報告中提到\"相關詞彙，避免修改專有名詞，避免做出總結，\
                    直接撰寫內容，避免回應要求。",
        info=contexts,
        model="openai:gpt-4"
    )
    result[title[0].split(": ")[-1]] = response
    contexts = []

    print(f"----------integrated summary of {title[0]}----------")
    print(response + "\n")

previous_result = ""
for value in result.values():
    previous_result += value

result["內容摘要"] = summary.summarize_articles(
    articles=previous_result,
    summary_len=500
)

total_time = time() - start_time

In [8]:
start_time = time()

def process_link(link: str, title: str, subtitle: str, theme: str) -> str:
    extract_topics = [f"{theme}相關的{title}", subtitle]
    return webpage_summary_func(
        url=link,
        summary_len=500,  # 假設我們想要500字的摘要
        extract_topics=extract_topics,
        list_result=False,
        model="openai:gpt-4",
        custom_prompts={
            'system_prompt': f"以{theme}為主題，請你總結撰寫出與\"{title}\"相關的內容，其中需包含{subtitle}，不需要結論，不需要回應要求。"
        }
    )


with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    for title, subtitle in titles.items():
        future_to_link = {executor.submit(process_link, link, title, subtitle, theme): link for link in links}
        title_contexts = []
        for future in concurrent.futures.as_completed(future_to_link):
            link = future_to_link[future]
            try:
                summary = future.result()
                if summary:
                    title_contexts.append(summary)
            except Exception as exc:
                print(f'{link} generated an exception: {exc}')

        if title_contexts:
            response = QA.ask_self(
                prompt=f"將此內容以客觀角度進行融合，避免使用\"報告中提到\"相關詞彙，避免修改專有名詞，避免做出總結，直接撰寫內容，避免回應要求。",
                info=title_contexts,
                model="openai:gpt-4"
            )
            result[title] = response
        else:
            result[title] = "無法獲取相關內容"

previous_result = ""
for value in result.values():
    previous_result += value

result["內容摘要"] = summary_class.summarize_articles(
    articles=previous_result,
    format_prompt=f"將內容以{theme}為主題進行摘要，將用字換句話說，意思不變，不需要結論，不需要回應要求。",
    summary_len=500
)
total_time = time() - start_time

selected openai model gpt-4.



Reduce_map Summary:   0%|          | 0/4 [00:00<?, ?it/s]

selected openai model gpt-4.

selected openai model gpt-4.



selected openai model gpt-4.



selected openai model gpt-4.



...

prompt: 
 
        先移除主文以外的文字，再進行重點論述擷取，最後以中文方式進行摘要。
        ---
        主文定義：文章的主要內容，不含頁首、尾、參考資料、廣告、網頁標籤等。
        ---
        重點論述擷取標的：
*
 
[
'
電
池
產
業
的
發
展
趨
勢
相
關
的
政
策
與
供
應
鏈
分
析
'
,
 
{
'
各
國
政
策
支
持
與
法
規
要
求
'
,
 
'
供
應
鏈
與
原
材
料
供
應
'
,
 
'
產
業
佈
局
與
投
資
趨
勢
'
,
 
'
環
保
與
回
收
政
策
'
}
]
若無法擷取上述標的，請放棄擷取，否則罰你10000元。
        擷取到的重點論述不來自主文，罰你10000元。
        ---
        摘要規範：
        * 摘要結果須為原文的保留或縮減，不可新增不存在於原文的內容
        * 請保留語句之人、事、時、地、物等專有名詞，不可刪除、更名或轉為代名詞
        * 數據、年份等數字需與原文完全相同，不可任意更改
        * 文意須盡可能對齊原文
        將以下內容總結成一個500字的摘要，一步一步來，如果你做得好，我會給你100美元小費。

------------
對不起，這段文字並未包含主文，因此無法進行摘要。
對不起，這篇文章並未提供電池產業的發展趨勢相關的政策與供應鏈分析，因此無法進行重點論述的擷取。
摘要：
全球至少136個國家宣示淨零碳排目標，推動了電池產業的發展。2022年全球新能源車銷售量約1065萬輛，年增63.6%，其中純電動車為789萬輛，年成長68.7%。中國寧德時代（CATL）在電池產業市占率達到37%，此後依序是比亞迪（BYD）、LG Energy Solutions兩者市占率為同為13.6%。台灣業者格斯科技董事長張忠傑認為，台灣的材料創新應用，可以讓電池芯在全球鋰電池產業關鍵地位，並希望透過對於電池材料的專業，切入鋰電池芯的製造。
格斯科技在2020年與日本上市公司LTO獲得電池代工試作契約，共同開發大容量LTO電池，並在2023年開始量產。台灣政府也推動儲能產業發展，有三大集團參與，包括能元科技、新智能科技和芯量科技。格斯科技強調，台灣業者

鋰鐵電池的市佔率將激增至30%。目前全球電動車動力電池的主要供應商為中國寧德時代、韓國LG和日本松下。台灣地區的鋰離子電池原材料大部分仍需進口，自主供應比重低於30%，而負極材料供應比重為15%。台灣電池產業正在積極提升國際競爭力，並尋求在新的電池業賽局中搶得商機。台灣在鋰電池產業布局近乎完整，並正致力於改良材料，研發鋰電池正極材料與電解液，以突破障礙。預計未來5到10年內固態電池將走向商業市場，而台灣也已投入研究，預計2030年將邁向全固態電池。prompt: 
 
        先移除主文以外的文字，再進行重點論述擷取，最後以中文方式進行摘要。
        ---
        主文定義：文章的主要內容，不含頁首、尾、參考資料、廣告、網頁標籤等。
        ---
        重點論述擷取標的：
*
 
[
'
電
池
產
業
的
發
展
趨
勢
相
關
的
政
策
與
供
應
鏈
分
析
'
,
 
{
'
各
國
政
策
支
持
與
法
規
要
求
'
,
 
'
供
應
鏈
與
原
材
料
供
應
'
,
 
'
產
業
佈
局
與
投
資
趨
勢
'
,
 
'
環
保
與
回
收
政
策
'
}
]
若無法擷取上述標的，請放棄擷取，否則罰你10000元。
        擷取到的重點論述不來自主文，罰你10000元。
        ---
        摘要規範：
        * 摘要結果須為原文的保留或縮減，不可新增不存在於原文的內容
        * 請保留語句之人、事、時、地、物等專有名詞，不可刪除、更名或轉為代名詞
        * 數據、年份等數字需與原文完全相同，不可任意更改
        * 文意須盡可能對齊原文
        將以下內容總結成一個500字的摘要，一步一步來，如果你做得好，我會給你100美元小費。

------------
能源元件產業主要指電池製造的上中下游關聯產業，包括電池原材料廠商、電池芯以及電池模組廠商。電池主要由電池芯、機殼與電源控制板三個元件組成。目前熟知的電池種類有鉛酸電池、鎳鎘電池、鎳氫電池、鋰電池、磷酸鐵鋰電池等，其中鋰電池是最被廣泛應用的電池技術之一。鉛酸電池是發展最早、生產最為成熟且量產的產品，主要應用於汽機車與工業設備的蓄電系統。鎳鎘電池能量密度與壽命較鉛酸電池好

Reduce_map Summary:  60%|██████    | 6/10 [02:53<01:55, 28.84s/it]

摘要：
能源元件產業主要包括電池原材料廠商、電池芯以及電池模組廠商。其中，鋰電池是最被廣泛應用的電池技術之一，並且鋰電池的發展是電池產業的重要趨勢。鋰離子電池因其高能量儲存量、低自放電率及長壽命，被廣泛應用在各種電子3C產品上。磷酸鋰鐵電池具有成本低、安全性高、壽命長等優點，且預估到2030年，磷酸鋰鐵電池的市佔率將激增至30%。目前全球電動車動力電池的主要供應商為中國寧德時代、韓國LG和日本松下。臺灣地區的鋰離子電池原材料大部分仍需進口，自主供應比重低於30%，而負極材料供應比重為15%。臺灣電池產業正在積極提升國際競爭力，並尋求在新的電池業賽局中搶得商機。臺灣在鋰電池產業佈局近乎完整，並正致力於改良材料，研發鋰電池正極材料與電解液，以突破障礙。預計未來5到10年內固態電池將走向商業市場，而臺灣也已投入研究，預計2030年將邁向全固態電池。 






KeyboardInterrupt: 

In [ ]:
result["內容摘要"] = summary.summarize_articles(
    articles=previous_result,
    format_prompt="將內容進行摘要，將用字換句話說，意思不便，不需要結論，不需要回應要求。",
    summary_len=500
)

In [7]:
print("----------Complete report----------")
# print("Total time: ", total_time, "s")
print("Total time: ")
for key, value in result.items():
    print(key, ":\n\n\t", value, "\n")

----------Complete report----------
Total time: 
全球電池市場概況 :

	 臺灣電池產業在電池模組領域表現優異，並有機會進一步發展動力電池模組和上游電池材料領域。然而，中游電池芯領域需要內需市場和國家政策的支持才能成功。全球鋰電池產業總產值將超過千億美元，對電動車和綠能產業的發展將產生重大影響。臺灣鋰電池廠商面臨中日韓對手的強大競爭，以及歐美國家的迅速追趕。

臺灣預計在2024年將有五座超級電芯廠開始量產，其中包括臺塑集團在彰濱工業區的超級電池芯工廠，以及鴻海的自有電池試產線。預估到2027年，臺灣的總產能將達到12~13GWh，佔全球總產量3300GWh的不到0.3%。儘管如此，臺灣仍認為電池芯在能源轉型中扮演關鍵角色。

鋰電池在未來五年內將快速成長，並在未來十年內主宰電池市場。2022年到2030年鋰電池市場將從700GWh成長到4,700 GWh，市值將達4000億美元，年增率達33%。主要成長區域為中國、歐洲與美國，其中中國將佔有鋰電池市場的45%到40%。

臺灣的鋰電池供應鏈包含上游的原材料、中游的電池芯和模組、以及下游的電池組與應用。然而，由於規模較小，上、中游廠商難以達到量產與成本優勢。臺灣鋰電池產業的最大限制因素是應用市場規模小且技術不夠成熟。預計到2023年，電動車、充電樁、大型儲能櫃將成為臺灣鋰電池的三大市場。

固態電池和鈦酸鋰電池成為新的儲能希望，前者具有高安全性和不漏液的優點，後者則有安全、壽命長的優點，但能量密度低。此外，新的燃料電池是一種發電裝置，具有低污染、低噪音、免充電、高效率、壽命長、適用範圍廣和可以分散式供電的特性，是目前最具發展前景的新能源方式。預計未來5到10年內固態電池將走向商業市場，而臺灣也已投入研究，預計2030年將邁向全固態電池。 

技術發展趨勢 :

	 臺灣有五座超級電芯廠將於今年開始量產，包括臺塑集團在彰濱工業區的電池芯工廠和鴻海的自有電池試產線。這些工廠的產能將滿足大量的電池需求，例如1GWh的產能相當於1.2萬臺特斯拉電動車的電池需求。然而，臺灣的電池芯產業也面臨挑戰，中國的電芯產業在過去十年間快速崛起，囊括全球八成以上的供應量，且因為供過於求，電池芯價格崩跌。

鋰電池在未來五年內將快速成長，並在未來十年內主宰電池市場。臺灣鋰電池供應鏈的市場發展與